In [2]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine, types, text

# Crear conexión con SQL Server usando autenticación de Windows
engine = create_engine('mssql+pyodbc://BIANCA\\SQLEXPRESS/PremiumDrinks?trusted_connection=yes&driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes')

# Leer los archivos CSV correspondientes a cada tabla
VentasFinal2016_df = pd.read_csv(r'C:\Users\bianc\OneDrive\Escritorio\proyectofinal\LimpiezaSalesFinal31122016')
PrecioCompra_df = pd.read_csv(r'C:\Users\bianc\OneDrive\Escritorio\proyectofinal\Limpieza2017PurchasePricesDec')
ComprasFinal2016_df = pd.read_csv(r'C:\Users\bianc\OneDrive\Escritorio\proyectofinal\LimpiezaPurchasesFINAL12312016')

# Obtener los valores únicos de la columna NúmeroProveedor en todas las tablas
proveedores_unicos = pd.concat([
    ComprasFinal2016_df[['NúmeroProveedor']].drop_duplicates(),
    VentasFinal2016_df[['NúmeroProveedor']].drop_duplicates(),
    PrecioCompra_df[['NúmeroProveedor']].drop_duplicates()
])

# Eliminar duplicados de la columna 'NúmeroProveedor' y remover valores nulos
proveedores_unicos = proveedores_unicos.drop_duplicates().dropna(subset=['NúmeroProveedor'])

# Crear la tabla PROVEEDOR en SQL Server
dtype_sql_proveedor = {
    'NúmeroProveedor': types.INTEGER  # Asumimos que NúmeroProveedor es un entero en todas las tablas
}

# Exportar los datos únicos a la tabla PROVEEDOR
proveedores_unicos.to_sql('PROVEEDOR', con=engine, if_exists='replace', index=False, dtype=dtype_sql_proveedor)

print("Tabla PROVEEDOR creada exitosamente con los datos únicos de NúmeroProveedor.")

with engine.connect() as connection:
    # Verificar si aún hay valores nulos
    null_count = connection.execute(text("SELECT COUNT(*) FROM PROVEEDOR WHERE NúmeroProveedor IS NULL")).scalar()
    
    if null_count > 0:
        print(f"Se encontraron {null_count} valores nulos en la tabla PROVEEDOR. Asignando valores por defecto.")
        # Actualizar los valores nulos en la columna NúmeroProveedor
        connection.execute(text("""
            UPDATE PROVEEDOR
            SET NúmeroProveedor = -1  -- Se puede usar un valor por defecto o ajustarlo según lo que tenga sentido en tu contexto
            WHERE NúmeroProveedor IS NULL;
        """))

    # Asegurarse de que la columna NúmeroProveedor no tenga valores nulos y sea NOT NULL
    connection.execute(text("""
        ALTER TABLE PROVEEDOR
        ALTER COLUMN NúmeroProveedor INT NOT NULL;
    """))

    # Agregar clave primaria a la tabla PROVEEDOR
    connection.execute(text("""
        ALTER TABLE PROVEEDOR
        ADD CONSTRAINT PK_PROVEEDOR PRIMARY KEY (NúmeroProveedor);
    """))

print("Clave primaria agregada exitosamente a la tabla PROVEEDOR.")

Tabla PROVEEDOR creada exitosamente con los datos únicos de NúmeroProveedor.
Clave primaria agregada exitosamente a la tabla PROVEEDOR.
